# Libraries

In [1]:
# Import BW25 packages. You'll notice the packages are imported individually, unlike a one-and-done import with BW2.
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa
import matplotlib.pyplot as plt 
from collections import defaultdict
import numpy as np
# import seaborn as sns

# 1. Support Functions

In [2]:
import config
from database_setup import find_activity_by_name_product_location
from lifecycle import run_comprehensive_lcia

## 1.1. Function

## 1.2. Visuals

In [3]:
def plot_combined_comparison_lcia_results_across_years(comparison_results, methods_list, activity_name, project_name):
    """
    Plot all LCIA results for different methods across years for multiple scenarios into a single image using subplots.
    
    Parameters:
    - comparison_results: A dictionary where keys are scenarios, then years, then LCIA results dictionaries.
    - methods_list: A list of tuples representing the impact assessment methods.
    - activity_name: The name of the activity being analyzed.
    - project_name: The name of the project.
    """

    num_methods = len(methods_list)
    num_cols = 3  # Number of columns for subplots
    num_rows = (num_methods + num_cols - 1) // num_cols  # Calculate the number of rows needed

    plt.figure(figsize=(15, 5 * num_rows))

    for i, method in enumerate(methods_list, 1):
        plt.subplot(num_rows, num_cols, i)

        # Plot results for each scenario in the same subplot
        for scenario, results_dict in comparison_results.items():
            years = sorted(results_dict.keys())
            scores = [results_dict[year][method] for year in years]
            plt.plot(years, scores, marker='o', linestyle='-', label=scenario)

        plt.xlabel('Year')
        plt.ylabel('Impact Score')
        plt.title(f'{method[1]} ({method[2]})')
        plt.legend(title="Scenario")
        plt.grid(True)

    # Add a main title that includes the activity name and project name
    plt.suptitle(f'LCIA Results Comparison for {activity_name}\nProject: {project_name}', fontsize=16, y=1.02)

    plt.tight_layout()
    plt.show()


## 1.3. Lists

# 2. Imports and declarations

In [4]:
project_name = "LNV-EI38-20241204"
bd.projects.set_current(project_name)

In [5]:
bd.databases

Databases dictionary with 32 objects, including:
	EI38_cutoff_remind_SSP1-Base_2020_VSI
	EI38_cutoff_remind_SSP1-Base_2020_baseline
	EI38_cutoff_remind_SSP1-Base_2025_VSI
	EI38_cutoff_remind_SSP1-Base_2025_baseline
	EI38_cutoff_remind_SSP1-Base_2030_VSI
	EI38_cutoff_remind_SSP1-Base_2030_baseline
	EI38_cutoff_remind_SSP1-Base_2035_VSI
	EI38_cutoff_remind_SSP1-Base_2035_baseline
	EI38_cutoff_remind_SSP1-Base_2040_VSI
	EI38_cutoff_remind_SSP1-Base_2040_baseline
Use `list(this object)` to get the complete list.

# 3. Synthesis of LCIA

In [6]:
def run_lcia_across_years(activity_name, methods_list, databases, reference_product=None, location=None):
    """
    Run LCIA across multiple years (databases) and store results.
    
    Parameters:
    - activity_name: The name of the activity to analyze.
    - methods_list: A list of tuples representing the impact assessment methods.
    - databases: A list of database names (one for each year).
    
    Returns:
    - results_dict: A dictionary where keys are years and values are LCIA results dictionaries.
    """

    results_dict = {}

    for db_name in databases:
        # Set the current project
        bd.projects.set_current(project_name)

        try:
            # Find the specific activity in the current database
            activity = find_activity_by_name_product_location(db_name, activity_name, reference_product, location)

        except ValueError as e:
            print(e)
            continue
    
        # Extract the year by finding the part that looks like a year

        # Urgently need to find a better way to do this:

        year = next(part for part in db_name.split('_') if part.isdigit() and len(part) == 4)

        # Run LCIA for the current year
        lca_results = run_comprehensive_lcia(activity, methods_list)

        # Store the results indexed by year
        results_dict[year] = lca_results

    return results_dict

In [7]:
def run_lcia_comparison(activity_name, methods_list, databases_list_dict, reference_product=None, location=None):
    """
    Run LCIA for a given activity across multiple sets of databases (e.g., baseline and energy scenarios).
    
    Parameters:
    - activity_name: The name of the activity to analyze.
    - methods_list: A list of tuples representing the impact assessment methods.
    - databases_list_dict: A dictionary where the key is the scenario (e.g., 'baseline', 'energy'),
      and the value is a list of database names (one for each year).
    
    Returns:
    - comparison_results: A nested dictionary where the first key is the scenario (e.g., 'baseline'),
      the second key is the year, and the value is the LCIA results dictionary.
    """

    comparison_results = {}

    for scenario, databases in databases_list_dict.items():
        print(f"Running LCIA for scenario: {scenario}")
        results_per_scenario = run_lcia_across_years(activity_name, methods_list, databases, reference_product, location)
        comparison_results[scenario] = results_per_scenario
    
    return comparison_results


In [ ]:
# Define the activity and scenarios
activity_name = 'smelting and refining of nickel concentrate, 7% Ni'
reference_product = 'nickel, class 1'
location ='CN'

databases_list_dict = {
    'SSP1 - baseline': config.db_remindSSP1_baseline,
    'SSP1 - VSI': config.db_remindSSP1_VSI
}

# Run the comparison
comparison_results = run_lcia_comparison(activity_name, config.recipe_midpoint_h, databases_list_dict, reference_product, location)

# Plot the comparison for all impact categories in ReCiPe Midpoint (H)
plot_combined_comparison_lcia_results_across_years(comparison_results, config.recipe_midpoint_h, activity_name, "SSP1 x SSP2")

Running LCIA for scenario: SSP1 - baseline


AttributeError: 'tuple' object has no attribute 'decode'

In [13]:
# Define the activity and scenarios
activity_name = 'smelting and refining of nickel concentrate, 7% Ni'
reference_product = 'nickel, class 1'
location ='CN'

In [15]:
results = run_lcia_across_years(activity_name, config.recipe_midpoint_h, config.db_remindSSP1_VSI, reference_product, location)

('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'): 0.07750822765466615
('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'): 0.00965282616314665
('ReCiPe Midpoint (H)', 'photochemical oxidant formation', 'POFP'): 0.3066192377206748
('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'): 52.811311637623284
('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'): 0.07562072896228957
('ReCiPe Midpoint (H)', 'climate change', 'GWP100'): 33.744005784218956
('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'): 0.08605031992238797
('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'): 1.2543751839850592
('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'): 0.037082414907402605
('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'): 15.33828060797242
('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'): 0.21057980365750614
('ReCiPe Midpoint (H)', 'water depletion', 'WDP'): 0.1098250887699514
('ReCiPe Midpoint (H)', 'urban land occupation', 'U

In [16]:
results

{'2025': defaultdict(float,
             {('ReCiPe Midpoint (H)',
               'terrestrial ecotoxicity',
               'TETPinf'): 0.07750822765466615,
              ('ReCiPe Midpoint (H)',
               'natural land transformation',
               'NLTP'): 0.00965282616314665,
              ('ReCiPe Midpoint (H)',
               'photochemical oxidant formation',
               'POFP'): 0.3066192377206748,
              ('ReCiPe Midpoint (H)',
               'human toxicity',
               'HTPinf'): 52.811311637623284,
              ('ReCiPe Midpoint (H)',
               'marine eutrophication',
               'MEP'): 0.07562072896228957,
              ('ReCiPe Midpoint (H)',
               'climate change',
               'GWP100'): 33.744005784218956,
              ('ReCiPe Midpoint (H)',
               'particulate matter formation',
               'PMFP'): 0.08605031992238797,
              ('ReCiPe Midpoint (H)',
               'agricultural land occupation',
            

In [17]:
results = run_lcia_across_years(activity_name, config.recipe_midpoint_h, config.db_remindSSP1_baseline, reference_product, location)

('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'): 0.07750927637695375
('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'): 0.009652782892836822
('ReCiPe Midpoint (H)', 'photochemical oxidant formation', 'POFP'): 0.3066196576762173
('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'): 52.811422813506496
('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'): 0.07562106908543412
('ReCiPe Midpoint (H)', 'climate change', 'GWP100'): 33.744153488147724
('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'): 0.08605058102784617
('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'): 1.2544249894389492
('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'): 0.0370824297228069
('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'): 15.338286634050343
('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'): 0.21058060842315665
('ReCiPe Midpoint (H)', 'water depletion', 'WDP'): 0.10982808993131248
('ReCiPe Midpoint (H)', 'urban land occupation', '

In [18]:
results

{'2025': defaultdict(float,
             {('ReCiPe Midpoint (H)',
               'terrestrial ecotoxicity',
               'TETPinf'): 0.07750927637695375,
              ('ReCiPe Midpoint (H)',
               'natural land transformation',
               'NLTP'): 0.009652782892836822,
              ('ReCiPe Midpoint (H)',
               'photochemical oxidant formation',
               'POFP'): 0.3066196576762173,
              ('ReCiPe Midpoint (H)',
               'human toxicity',
               'HTPinf'): 52.811422813506496,
              ('ReCiPe Midpoint (H)',
               'marine eutrophication',
               'MEP'): 0.07562106908543412,
              ('ReCiPe Midpoint (H)',
               'climate change',
               'GWP100'): 33.744153488147724,
              ('ReCiPe Midpoint (H)',
               'particulate matter formation',
               'PMFP'): 0.08605058102784617,
              ('ReCiPe Midpoint (H)',
               'agricultural land occupation',
           